<a href="https://colab.research.google.com/github/hbbalamsyah/Currency-Nominal-Detection-Using-CNN/blob/main/Currency_Nominal_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install tensorflow

  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import pandas as pd
import h5py
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Dataset yang Telah di Preprocessing

In [ ]:
dataset = '/content/drive/MyDrive/dataset uang baru/5. resize'

Modelling

In [ ]:
# Load pre-trained VGG16 model tanpa lapisan fully connected
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Tambahkan lapisan fully connected custom
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

# Buat model akhir dengan pre-trained VGG16 sebagai base
model = Model(inputs=base_model.input, outputs=output)

# Kunci lapisan base agar tidak ikut terlatih ulang
for layer in base_model.layers:
    layer.trainable = False

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


58889256/58889256 [==============================] - 0s 0us/step


Training Model

In [ ]:
# Preprocessing data menggunakan ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # validation_split 20%

# Pembuatan data generator untuk pelatihan
train_generator = datagen.flow_from_directory(
    dataset,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Gunakan data pelatihan
)

# Pembuatan data generator untuk validasi
validation_generator = datagen.flow_from_directory(
    dataset,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Gunakan data validasi
)

# Pelatihan model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Simpan model
model.save('vgg16_rupiah_detection_model_epoch 10.h5')

Found 896 images belonging to 7 classes.
Found 224 images belonging to 7 classes.
Epoch 1/10
28/28 [==============================] - 879s 31s/step - loss: 2.8539 - accuracy: 0.2478 - val_loss: 1.6621 - val_accuracy: 0.2946
Epoch 2/10
28/28 [==============================] - 911s 33s/step - loss: 0.8958 - accuracy: 0.7254 - val_loss: 1.1398 - val_accuracy: 0.5804
Epoch 3/10
28/28 [==============================] - 928s 33s/step - loss: 0.4177 - accuracy: 0.9554 - val_loss: 0.8953 - val_accuracy: 0.7321
Epoch 4/10
28/28 [==============================] - 924s 33s/step - loss: 0.2197 - accuracy: 0.9944 - val_loss: 0.8332 - val_accuracy: 0.7188
Epoch 5/10
28/28 [==============================] - 927s 33s/step - loss: 0.1343 - accuracy: 1.0000 - val_loss: 0.7230 - val_accuracy: 0.7679
Epoch 6/10
28/28 [==============================] - 922s 33s/step - loss: 0.0846 - accuracy: 1.0000 - val_loss: 0.6626 - val_accuracy: 0.7857
Epoch 7/10
28/28 [==============================] - 896s 32s/step 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Mengambil labelling yang di dapat pada proses training**

Setelah diambil labellingnya, simpan label ke dalam file H5, agar tidak perlu running ulang code

In [ ]:
# Ambil label kelas dari generator pelatihan
class_labels = list(train_generator.class_indices.keys())

# Simpan label kelas ke dalam file H5
h5_file_path = '/content/class_labels.h5'  # Sesuaikan dengan lokasi penyimpanan file H5
with h5py.File(h5_file_path, 'w') as h5_file:
    h5_file.create_dataset('class_labels', data=np.array(class_labels, dtype='S'))


Load file H5 Labelling yang telah disimpan

In [ ]:
# Load file H5 Labelling
h5_file_path = '/content/drive/MyDrive/ujicoba/class_labels.h5'  # Sesuaikan path dengan lokasi file H5

# Buat dataset untuk mendapatkan label kelas
with h5py.File(h5_file_path, 'r') as h5_file:
    class_labels = h5_file['class_labels'][:]  # Sesuaikan dengan nama dataset yang sesuai dalam file H5

# Ubah data numpy menjadi list
class_labels = class_labels.tolist()

# Print label kelas
print("Class Labels:", class_labels)


Class Labels: [b'100k', b'10k', b'1k', b'20k', b'2k', b'50k', b'5k']


**Import data uji**

In [ ]:
#import data uji

dataset_uji_terang = '/content/drive/MyDrive/uang uji baru/Terang_125'
dataset_uji_normal = '/content/drive/MyDrive/uang uji all'
dataset_uji_gelap = '/content/drive/MyDrive/uang uji baru/Gelap_075'

**Testing/Pengujian data all dalam 1 folder**

In [ ]:
# Load model yang telah dilatih
model = tf.keras.models.load_model('/content/drive/MyDrive/ujicoba/vgg16_rupiah_detection_model_epoch 5.h5')

# Ambil label kelas dari generator pelatihan

#class_labels = list(train_generator.class_indices.keys()) # Sudah diambil dari file berformat H5, jika belum ada baru di running

# Direktori tempat menyimpan gambar-gambar uji
dataset_uji_dir = dataset_uji_terang #ganti path jika mau uji dengan kondisi yang berbeda

# List semua nama file gambar dalam direktori uji
gambar_uji_filenames = os.listdir(dataset_uji_dir)

# Lakukan prediksi pada setiap gambar uji
for i, gambar_filename in enumerate(gambar_uji_filenames):
    gambar_path = os.path.join(dataset_uji_dir, gambar_filename)
    gambar = load_img(gambar_path, target_size=(256, 256))
    gambar_array = img_to_array(gambar)
    gambar_array = gambar_array.reshape((1, 256, 256, 3))
    gambar_array /= 255.0  # Normalisasi intensitas
    prediction = model.predict(gambar_array)
    predicted_class = class_labels[prediction.argmax()]
    print(f"{gambar_filename} : : {predicted_class}")

1/1 [==============================] - 1s 763ms/step
Copy of 1k4.JPG : : b'1k'
1/1 [==============================] - 1s 620ms/step
Copy of 2k4.JPG : : b'2k'
1/1 [==============================] - 1s 610ms/step
Copy of 5k4.JPG : : b'5k'
1/1 [==============================] - 1s 608ms/step
Copy of 10k4.JPG : : b'10k'
1/1 [==============================] - 1s 603ms/step
Copy of 20k4.JPG : : b'20k'
1/1 [==============================] - 1s 606ms/step
Copy of 50k4.JPG : : b'50k'
1/1 [==============================] - 1s 614ms/step
Copy of 100k4.JPG : : b'100k'


**Testing/Pengujian data per folder**

In [ ]:
# Load model yang telah dilatih
model = tf.keras.models.load_model('/content/vgg16_rupiah_detection_model_epoch 5.h5')

# Preprocessing data uji menggunakan ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Pembuatan data generator untuk uji
test_generator = datagen.flow_from_directory(
    dataset_uji_terang,  # Ganti dengan path sesuai direktori data mana yang akan diuji
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,  # Mode ini memungkinkan model untuk menghasilkan prediksi
    shuffle=False  # Jangan acak urutan gambar
)

# Prediksi menggunakan model
predictions = model.predict(test_generator)

# Label kelas
class_labels = list(train_generator.class_indices.keys())

# Menampilkan hasil prediksi dengan nama file
for i, prediction in enumerate(predictions):
    gambar_filename = os.path.basename(test_generator.filenames[i])
    predicted_class = class_labels[prediction.argmax()]
    print(f"{gambar_filename} : : {predicted_class}")

Found 28 images belonging to 7 classes.
1/1 [==============================] - 22s 22s/step
100k1.JPG : : 100k
100k2.JPG : : 100k
100k3.JPG : : 100k
100k4.JPG : : 100k
10k1.JPG : : 10k
10k2.JPG : : 50k
10k3.JPG : : 10k
10k4.JPG : : 10k
1k1.JPG : : 1k
1k2.JPG : : 1k
1k3.JPG : : 1k
1k4.JPG : : 1k
20k1.JPG : : 20k
20k2.JPG : : 20k
20k3.JPG : : 20k
20k4.JPG : : 20k
2k1.JPG : : 2k
2k2.JPG : : 2k
2k3.JPG : : 2k
2k4.JPG : : 2k
50k1.JPG : : 50k
50k2.JPG : : 50k
50k3.JPG : : 50k
50k4.JPG : : 50k
5k1.JPG : : 5k
5k2.JPG : : 5k
5k3.JPG : : 5k
5k4.JPG : : 5k
